<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2: Singapore Housing Data and Kaggle Challenge

_SG-DSI-41 Group 01: Daryl Chia, Germaine Choo, Lionel Foo_

---

## Generating Predictions for Kaggle

## 01. Import Libraries

In [215]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# sklearn imports
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn import metrics

## 02. Optimal Model: Model 3 - Polynomial Features  (Adjusted for Multicollinearity)

### a. Load Data

In [216]:
# Load data:
model_3 = pd.read_csv('./output/model_3.csv')

# Check data:
print(model_3.shape)
model_3.head()

(149805, 504)


,Unnamed: 0,2room_sold,3room_sold,floor_area_sqft,hawker_food_stalls,vacancy,hdb_age^2,hdb_age max_floor_lvl,hdb_age 2room_sold,hdb_age 4room_sold,hdb_age 1room_rental,hdb_age mid_storey,hdb_age floor_area_sqft,hdb_age bus_stop_nearest_distance,hdb_age mrt_nearest_distance,hdb_age mall_nearest_distance,hdb_age mall_within_1km,hdb_age mall_within_2km,hdb_age hawker_food_stalls,hdb_age hawker_market_stalls,hdb_age hawker_nearest_distance,hdb_age hawker_within_500m,hdb_age vacancy,hdb_age pri_sch_nearest_distance,hdb_age sec_sch_nearest_dist,hdb_age 1room_sold,max_floor_lvl 4room_sold,max_floor_lvl 5room_sold,max_floor_lvl floor_area_sqft,max_floor_lvl bus_stop_nearest_distance,max_floor_lvl mall_nearest_distance,max_floor_lvl mall_within_500m,max_floor_lvl mall_within_1km,max_floor_lvl mall_within_2km,max_floor_lvl hawker_food_stalls,max_floor_lvl hawker_market_stalls,max_floor_lvl hawker_within_500m,max_floor_lvl hawker_within_2km,max_floor_lvl vacancy,max_floor_lvl pri_sch_nearest_distance,max_floor_lvl sec_sch_nearest_dist,total_dwelling_units floor_area_sqft,total_dwelling_units bus_stop_nearest_distance,total_dwelling_units mrt_nearest_distance,total_dwelling_units mall_nearest_distance,total_dwelling_units mall_within_2km,total_dwelling_units hawker_food_stalls,total_dwelling_units hawker_nearest_distance,total_dwelling_units pri_sch_nearest_distance,total_dwelling_units cutoff_point,total_dwelling_units sec_sch_nearest_dist,2room_sold 3room_sold,2room_sold 4room_sold,2room_sold 5room_sold,2room_sold exec_sold,2room_sold multigen_sold,2room_sold 2room_rental,2room_sold 3room_rental,2room_sold other_room_rental,2room_sold mall_within_500m,2room_sold mall_within_2km,2room_sold hawker_market_stalls,2room_sold hawker_nearest_distance,2room_sold 1room_sold,3room_sold 4room_sold,3room_sold 5room_sold,3room_sold exec_sold,3room_sold multigen_sold,3room_sold studio_apartment_sold,3room_sold 1room_rental,3room_sold other_room_rental,3room_sold mall_within_500m,3room_sold mall_within_1km,3room_sold mall_within_2km,3room_sold hawker_nearest_distance,3room_sold hawker_within_500m,4room_sold 5room_sold,4room_sold exec_sold,4room_sold studio_apartment_sold,4room_sold 2room_rental,4room_sold 3room_rental,4room_sold bus_stop_nearest_distance,4room_sold mrt_nearest_distance,4room_sold mall_nearest_distance,4room_sold mall_within_500m,4room_sold mall_within_1km,4room_sold mall_within_2km,4room_sold hawker_food_stalls,4room_sold hawker_market_stalls,4room_sold hawker_nearest_distance,4room_sold hawker_within_500m,4room_sold hawker_within_2km,4room_sold vacancy,4room_sold pri_sch_nearest_distance,4room_sold sec_sch_nearest_dist,4room_sold 1room_sold,5room_sold exec_sold,5room_sold multigen_sold,5room_sold studio_apartment_sold,5room_sold 1room_rental,5room_sold 2room_rental,5room_sold 3room_rental,5room_sold other_room_rental,5room_sold bus_stop_nearest_distance,5room_sold mrt_nearest_distance,5room_sold mall_nearest_distance,5room_sold mall_within_500m,5room_sold mall_within_1km,5room_sold hawker_food_stalls,5room_sold hawker_market_stalls,5room_sold hawker_nearest_distance,5room_sold hawker_within_500m,5room_sold hawker_within_2km,5room_sold pri_sch_nearest_distance,5room_sold sec_sch_nearest_dist,5room_sold 1room_sold,exec_sold multigen_sold,exec_sold studio_apartment_sold,exec_sold 1room_rental,exec_sold 2room_rental,exec_sold 3room_rental,exec_sold other_room_rental,exec_sold floor_area_sqft,exec_sold mall_within_500m,exec_sold mall_within_1km,exec_sold hawker_market_stalls,exec_sold hawker_nearest_distance,exec_sold hawker_within_500m,exec_sold hawker_within_2km,exec_sold 1room_sold,multigen_sold studio_apartment_sold,multigen_sold 1room_rental,multigen_sold 2room_rental,multigen_sold 3room_rental,multigen_sold other_room_rental,multigen_sold floor_area_sqft,multigen_sold mall_nearest_distance,multigen_sold hawker_market_stalls,multigen_sold 1room_sold,studio_apartment_sold 1room_rental,studio_apartment_sold 2room_rental,studio_apartmen

In [217]:
# Drop the 'Unnamed: 0' column from Model_3
model_3.drop(columns = ['Unnamed: 0'], inplace = True)


### b. Separate Numerical and Categorical Features

In [218]:
# Identify binary categorical columns
model_3_binary_columns = ['commercial',
 'market_hawker',
 'multistorey_carpark',
 'precinct_pavilion',
 'bus_interchange',
 'mrt_interchange',
 'pri_sch_affiliation',
 'affiliation',
 'tranc_yearmonth_2012-04',
 'tranc_yearmonth_2012-05',
 'tranc_yearmonth_2012-06',
 'tranc_yearmonth_2012-07',
 'tranc_yearmonth_2012-08',
 'tranc_yearmonth_2012-09',
 'tranc_yearmonth_2012-10',
 'tranc_yearmonth_2012-11',
 'tranc_yearmonth_2012-12',
 'tranc_yearmonth_2013-01',
 'tranc_yearmonth_2013-02',
 'tranc_yearmonth_2013-03',
 'tranc_yearmonth_2013-04',
 'tranc_yearmonth_2013-05',
 'tranc_yearmonth_2013-06',
 'tranc_yearmonth_2013-07',
 'tranc_yearmonth_2013-08',
 'tranc_yearmonth_2013-09',
 'tranc_yearmonth_2013-10',
 'tranc_yearmonth_2013-11',
 'tranc_yearmonth_2013-12',
 'tranc_yearmonth_2014-01',
 'tranc_yearmonth_2014-02',
 'tranc_yearmonth_2014-03',
 'tranc_yearmonth_2014-04',
 'tranc_yearmonth_2014-05',
 'tranc_yearmonth_2014-06',
 'tranc_yearmonth_2014-07',
 'tranc_yearmonth_2014-08',
 'tranc_yearmonth_2014-09',
 'tranc_yearmonth_2014-10',
 'tranc_yearmonth_2014-11',
 'tranc_yearmonth_2014-12',
 'tranc_yearmonth_2015-01',
 'tranc_yearmonth_2015-02',
 'tranc_yearmonth_2015-03',
 'tranc_yearmonth_2015-04',
 'tranc_yearmonth_2015-05',
 'tranc_yearmonth_2015-06',
 'tranc_yearmonth_2015-07',
 'tranc_yearmonth_2015-08',
 'tranc_yearmonth_2015-09',
 'tranc_yearmonth_2015-10',
 'tranc_yearmonth_2015-11',
 'tranc_yearmonth_2015-12',
 'tranc_yearmonth_2016-01',
 'tranc_yearmonth_2016-02',
 'tranc_yearmonth_2016-03',
 'tranc_yearmonth_2016-04',
 'tranc_yearmonth_2016-05',
 'tranc_yearmonth_2016-06',
 'tranc_yearmonth_2016-07',
 'tranc_yearmonth_2016-08',
 'tranc_yearmonth_2016-09',
 'tranc_yearmonth_2016-10',
 'tranc_yearmonth_2016-11',
 'tranc_yearmonth_2016-12',
 'tranc_yearmonth_2017-01',
 'tranc_yearmonth_2017-02',
 'tranc_yearmonth_2017-03',
 'tranc_yearmonth_2017-04',
 'tranc_yearmonth_2017-05',
 'tranc_yearmonth_2017-06',
 'tranc_yearmonth_2017-07',
 'tranc_yearmonth_2017-08',
 'tranc_yearmonth_2017-09',
 'tranc_yearmonth_2017-10',
 'tranc_yearmonth_2017-11',
 'tranc_yearmonth_2017-12',
 'tranc_yearmonth_2018-01',
 'tranc_yearmonth_2018-02',
 'tranc_yearmonth_2018-03',
 'tranc_yearmonth_2018-04',
 'tranc_yearmonth_2018-05',
 'tranc_yearmonth_2018-06',
 'tranc_yearmonth_2018-07',
 'tranc_yearmonth_2018-08',
 'tranc_yearmonth_2018-09',
 'tranc_yearmonth_2018-10',
 'tranc_yearmonth_2018-11',
 'tranc_yearmonth_2018-12',
 'tranc_yearmonth_2019-01',
 'tranc_yearmonth_2019-02',
 'tranc_yearmonth_2019-03',
 'tranc_yearmonth_2019-04',
 'tranc_yearmonth_2019-05',
 'tranc_yearmonth_2019-06',
 'tranc_yearmonth_2019-07',
 'tranc_yearmonth_2019-08',
 'tranc_yearmonth_2019-09',
 'tranc_yearmonth_2019-10',
 'tranc_yearmonth_2019-11',
 'tranc_yearmonth_2019-12',
 'tranc_yearmonth_2020-01',
 'tranc_yearmonth_2020-02',
 'tranc_yearmonth_2020-03',
 'tranc_yearmonth_2020-04',
 'tranc_yearmonth_2020-05',
 'tranc_yearmonth_2020-06',
 'tranc_yearmonth_2020-07',
 'tranc_yearmonth_2020-08',
 'tranc_yearmonth_2020-09',
 'tranc_yearmonth_2020-10',
 'tranc_yearmonth_2020-11',
 'tranc_yearmonth_2020-12',
 'tranc_yearmonth_2021-01',
 'tranc_yearmonth_2021-02',
 'tranc_yearmonth_2021-03',
 'tranc_yearmonth_2021-04',
 'planning_area_Bedok',
 'planning_area_Bishan',
 'planning_area_Bukit Batok',
 'planning_area_Bukit Merah',
 'planning_area_Bukit Panjang',
 'planning_area_Bukit Timah',
 'planning_area_Changi',
 'planning_area_Choa Chu Kang',
 'planning_area_Clementi',
 'planning_area_Downtown Core',
 'planning_area_Geylang',
 'planning_area_Hougang',
 'planning_area_Jurong East',
 'planning_area_Jurong West',
 'planning_area_Kallang',
 'planning_area_Marine Parade',
 'planning_area_Novena',
 'planning_area_Outram',
 'planning_area_Pasir Ris',
 'planning_area_Punggol',
 'planning_area_Queenstown',
 'planning_area_Rochor',
 'planning_area_Sembawang',
 'planning_area_Sengkang',
 'planning_area_Serangoon',
 'planning_area_Tampines',
 'planning_area_Tanglin',
 'planning_area_Toa Payoh',
 'planning_area_Western Water Catchment',
 'planning_area_Woodlands',
 'planning_area_Yishun',
 'flat_type_2 ROOM',
 'flat_type_3 ROOM',
 'flat_type_4 ROOM',
 'flat_type_5 ROOM',
 'flat_type_EXECUTIVE',
 'flat_type_MULTI-GENERATION',
 'flat_model_Adjoined flat',
 'flat_model_Apartment',
 'flat_model_DBSS',
 'flat_model_Improved',
 'flat_model_Improved-Maisonette',
 'flat_model_Maisonette',
 'flat_model_Model A',
 'flat_model_Model A-Maisonette',
 'flat_model_Model A2',
 'flat_model_Multi Generation',
 'flat_model_New Generation',
 'flat_model_Premium Apartment',
 'flat_model_Premium Apartment Loft',
 'flat_model_Premium Maisonette',
 'flat_model_Simplified',
 'flat_model_Standard',
 'flat_model_Terrace',
 'flat_model_Type S1',
 'flat_model_Type S2',
 'full_flat_type_2 ROOM 2-room',
 'full_flat_type_2 ROOM DBSS',
 'full_flat_type_2 ROOM Improved',
 'full_flat_type_2 ROOM Model A',
 'full_flat_type_2 ROOM Premium Apartment',
 'full_flat_type_2 ROOM Standard',
 'full_flat_type_3 ROOM DBSS',
 'full_flat_type_3 ROOM Improved',
 'full_flat_type_3 ROOM Model A',
 'full_flat_type_3 ROOM New Generation',
 'full_flat_type_3 ROOM Premium Apartment',
 'full_flat_type_3 ROOM Simplified',
 'full_flat_type_3 ROOM Standard',
 'full_flat_type_3 ROOM Terrace',
 'full_flat_type_4 ROOM Adjoined flat',
 'full_flat_type_4 ROOM DBSS',
 'full_flat_type_4 ROOM Improved',
 'full_flat_type_4 ROOM Model A',
 'full_flat_type_4 ROOM Model A2',
 'full_flat_type_4 ROOM New Generation',
 'full_flat_type_4 ROOM Premium Apartment',
 'full_flat_type_4 ROOM Premium Apartment Loft',
 'full_flat_type_4 ROOM Simplified',
 'full_flat_type_4 ROOM Standard',
 'full_flat_type_4 ROOM Terrace',
 'full_flat_type_4 ROOM Type S1',
 'full_flat_type_5 ROOM Adjoined flat',
 'full_flat_type_5 ROOM DBSS',
 'full_flat_type_5 ROOM Improved',
 'full_flat_type_5 ROOM Improved-Maisonette',
 'full_flat_type_5 ROOM Model A',
 'full_flat_type_5 ROOM Model A-Maisonette',
 'full_flat_type_5 ROOM Premium Apartment',
 'full_flat_type_5 ROOM Premium Apartment Loft',
 'full_flat_type_5 ROOM Standard',
 'full_flat_type_5 ROOM Type S2',
 'full_flat_type_EXECUTIVE Adjoined flat',
 'full_flat_type_EXECUTIVE Apartment',
 'full_flat_type_EXECUTIVE Maisonette',
 'full_flat_type_EXECUTIVE Premium Apartment',
 'full_flat_type_EXECUTIVE Premium Maisonette',
 'full_flat_type_MULTI-GENERATION Multi Generation',
 'tranc_year_2013',
 'tranc_year_2014',
 'tranc_year_2015',
 'tranc_year_2016',
 'tranc_year_2017',
 'tranc_year_2018',
 'tranc_year_2019',
 'tranc_year_2020',
 'tranc_year_2021',
 'tranc_month_10',
 'tranc_month_11',
 'tranc_month_12',
 'tranc_month_2',
 'tranc_month_3',
 'tranc_month_4',
 'tranc_month_5',
 'tranc_month_6',
 'tranc_month_7',
 'tranc_month_8',
 'tranc_month_9']


model_3_binary_columns

['commercial',
 'market_hawker',
 'multistorey_carpark',
 'precinct_pavilion',
 'bus_interchange',
 'mrt_interchange',
 'pri_sch_affiliation',
 'affiliation',
 'tranc_yearmonth_2012-04',
 'tranc_yearmonth_2012-05',
 'tranc_yearmonth_2012-06',
 'tranc_yearmonth_2012-07',
 'tranc_yearmonth_2012-08',
 'tranc_yearmonth_2012-09',
 'tranc_yearmonth_2012-10',
 'tranc_yearmonth_2012-11',
 'tranc_yearmonth_2012-12',
 'tranc_yearmonth_2013-01',
 'tranc_yearmonth_2013-02',
 'tranc_yearmonth_2013-03',
 'tranc_yearmonth_2013-04',
 'tranc_yearmonth_2013-05',
 'tranc_yearmonth_2013-06',
 'tranc_yearmonth_2013-07',
 'tranc_yearmonth_2013-08',
 'tranc_yearmonth_2013-09',
 'tranc_yearmonth_2013-10',
 'tranc_yearmonth_2013-11',
 'tranc_yearmonth_2013-12',
 'tranc_yearmonth_2014-01',
 'tranc_yearmonth_2014-02',
 'tranc_yearmonth_2014-03',
 'tranc_yearmonth_2014-04',
 'tranc_yearmonth_2014-05',
 'tranc_yearmonth_2014-06',
 'tranc_yearmonth_2014-07',
 'tranc_yearmonth_2014-08',
 'tranc_yearmonth_2014-09',
 

In [219]:
# Check binary columns 
model_3_binary_columns

['commercial',
 'market_hawker',
 'multistorey_carpark',
 'precinct_pavilion',
 'bus_interchange',
 'mrt_interchange',
 'pri_sch_affiliation',
 'affiliation',
 'tranc_yearmonth_2012-04',
 'tranc_yearmonth_2012-05',
 'tranc_yearmonth_2012-06',
 'tranc_yearmonth_2012-07',
 'tranc_yearmonth_2012-08',
 'tranc_yearmonth_2012-09',
 'tranc_yearmonth_2012-10',
 'tranc_yearmonth_2012-11',
 'tranc_yearmonth_2012-12',
 'tranc_yearmonth_2013-01',
 'tranc_yearmonth_2013-02',
 'tranc_yearmonth_2013-03',
 'tranc_yearmonth_2013-04',
 'tranc_yearmonth_2013-05',
 'tranc_yearmonth_2013-06',
 'tranc_yearmonth_2013-07',
 'tranc_yearmonth_2013-08',
 'tranc_yearmonth_2013-09',
 'tranc_yearmonth_2013-10',
 'tranc_yearmonth_2013-11',
 'tranc_yearmonth_2013-12',
 'tranc_yearmonth_2014-01',
 'tranc_yearmonth_2014-02',
 'tranc_yearmonth_2014-03',
 'tranc_yearmonth_2014-04',
 'tranc_yearmonth_2014-05',
 'tranc_yearmonth_2014-06',
 'tranc_yearmonth_2014-07',
 'tranc_yearmonth_2014-08',
 'tranc_yearmonth_2014-09',
 

In [220]:
# Identify numerical columns
model_3_numerical_columns = model_3.columns.copy()
model_3_numerical_columns = [i for i in model_3_numerical_columns if i not in model_3_binary_columns]

In [221]:
# Check binary columns 
model_3_binary_columns

['commercial',
 'market_hawker',
 'multistorey_carpark',
 'precinct_pavilion',
 'bus_interchange',
 'mrt_interchange',
 'pri_sch_affiliation',
 'affiliation',
 'tranc_yearmonth_2012-04',
 'tranc_yearmonth_2012-05',
 'tranc_yearmonth_2012-06',
 'tranc_yearmonth_2012-07',
 'tranc_yearmonth_2012-08',
 'tranc_yearmonth_2012-09',
 'tranc_yearmonth_2012-10',
 'tranc_yearmonth_2012-11',
 'tranc_yearmonth_2012-12',
 'tranc_yearmonth_2013-01',
 'tranc_yearmonth_2013-02',
 'tranc_yearmonth_2013-03',
 'tranc_yearmonth_2013-04',
 'tranc_yearmonth_2013-05',
 'tranc_yearmonth_2013-06',
 'tranc_yearmonth_2013-07',
 'tranc_yearmonth_2013-08',
 'tranc_yearmonth_2013-09',
 'tranc_yearmonth_2013-10',
 'tranc_yearmonth_2013-11',
 'tranc_yearmonth_2013-12',
 'tranc_yearmonth_2014-01',
 'tranc_yearmonth_2014-02',
 'tranc_yearmonth_2014-03',
 'tranc_yearmonth_2014-04',
 'tranc_yearmonth_2014-05',
 'tranc_yearmonth_2014-06',
 'tranc_yearmonth_2014-07',
 'tranc_yearmonth_2014-08',
 'tranc_yearmonth_2014-09',
 

In [222]:
# Check binary columns
model_3_numerical_columns

['2room_sold',
 '3room_sold',
 'floor_area_sqft',
 'hawker_food_stalls',
 'vacancy',
 'hdb_age^2',
 'hdb_age max_floor_lvl',
 'hdb_age 2room_sold',
 'hdb_age 4room_sold',
 'hdb_age 1room_rental',
 'hdb_age mid_storey',
 'hdb_age floor_area_sqft',
 'hdb_age bus_stop_nearest_distance',
 'hdb_age mrt_nearest_distance',
 'hdb_age mall_nearest_distance',
 'hdb_age mall_within_1km',
 'hdb_age mall_within_2km',
 'hdb_age hawker_food_stalls',
 'hdb_age hawker_market_stalls',
 'hdb_age hawker_nearest_distance',
 'hdb_age hawker_within_500m',
 'hdb_age vacancy',
 'hdb_age pri_sch_nearest_distance',
 'hdb_age sec_sch_nearest_dist',
 'hdb_age 1room_sold',
 'max_floor_lvl 4room_sold',
 'max_floor_lvl 5room_sold',
 'max_floor_lvl floor_area_sqft',
 'max_floor_lvl bus_stop_nearest_distance',
 'max_floor_lvl mall_nearest_distance',
 'max_floor_lvl mall_within_500m',
 'max_floor_lvl mall_within_1km',
 'max_floor_lvl mall_within_2km',
 'max_floor_lvl hawker_food_stalls',
 'max_floor_lvl hawker_market_st

In [223]:
# Drop target variable 'resale_price'from numerical columns
model_3_numerical_columns.remove('resale_price')

# check numerical columns
model_3_numerical_columns

['2room_sold',
 '3room_sold',
 'floor_area_sqft',
 'hawker_food_stalls',
 'vacancy',
 'hdb_age^2',
 'hdb_age max_floor_lvl',
 'hdb_age 2room_sold',
 'hdb_age 4room_sold',
 'hdb_age 1room_rental',
 'hdb_age mid_storey',
 'hdb_age floor_area_sqft',
 'hdb_age bus_stop_nearest_distance',
 'hdb_age mrt_nearest_distance',
 'hdb_age mall_nearest_distance',
 'hdb_age mall_within_1km',
 'hdb_age mall_within_2km',
 'hdb_age hawker_food_stalls',
 'hdb_age hawker_market_stalls',
 'hdb_age hawker_nearest_distance',
 'hdb_age hawker_within_500m',
 'hdb_age vacancy',
 'hdb_age pri_sch_nearest_distance',
 'hdb_age sec_sch_nearest_dist',
 'hdb_age 1room_sold',
 'max_floor_lvl 4room_sold',
 'max_floor_lvl 5room_sold',
 'max_floor_lvl floor_area_sqft',
 'max_floor_lvl bus_stop_nearest_distance',
 'max_floor_lvl mall_nearest_distance',
 'max_floor_lvl mall_within_500m',
 'max_floor_lvl mall_within_1km',
 'max_floor_lvl mall_within_2km',
 'max_floor_lvl hawker_food_stalls',
 'max_floor_lvl hawker_market_st

In [224]:
# Check total count of columns
len(model_3_binary_columns + model_3_numerical_columns)

502

In [225]:
# Separate binary and numerical columns
model_3_binary_data = model_3[model_3_binary_columns]
model_3_numerical_data = model_3[model_3_numerical_columns]

### c. Scale numerical data

In [226]:
# Standardize the numerical features using StandardScaler
ss3 = StandardScaler()
model_3_numerical_data_scaled = ss3.fit_transform(model_3_numerical_data)

### d. Creating Features (X) and Target Variable (y)

In [227]:
# Concatenate scaled numerical columns with binary columns
model_3_combined_data = pd.concat([pd.DataFrame(model_3_numerical_data_scaled, columns=model_3_numerical_columns), model_3_binary_data], axis=1)

In [228]:
# Separate the target variable 'resale_price' (y) and features (X) from model_3
y3 = model_3['resale_price']
X3 = model_3_combined_data

In [229]:
# Split the dataset into training and testing sets
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.2, random_state=42)

### e. Model Development

In [230]:
# Instantiate a Ridge Regression model
r_alphas = np.logspace(-2, 2, 100)
ridge3 = RidgeCV(alphas = r_alphas)

In [231]:
# Fit the Ridge model to the training data
ridge3.fit(X_train3, y_train3)

RidgeCV(alphas=array([1.00000000e-02, 1.09749877e-02, 1.20450354e-02, 1.32194115e-02,
       1.45082878e-02, 1.59228279e-02, 1.74752840e-02, 1.91791026e-02,
       2.10490414e-02, 2.31012970e-02, 2.53536449e-02, 2.78255940e-02,
       3.05385551e-02, 3.35160265e-02, 3.67837977e-02, 4.03701726e-02,
       4.43062146e-02, 4.86260158e-02, 5.33669923e-02, 5.85702082e-02,
       6.42807312e-02, 7.05480231e-0...
       1.17681195e+01, 1.29154967e+01, 1.41747416e+01, 1.55567614e+01,
       1.70735265e+01, 1.87381742e+01, 2.05651231e+01, 2.25701972e+01,
       2.47707636e+01, 2.71858824e+01, 2.98364724e+01, 3.27454916e+01,
       3.59381366e+01, 3.94420606e+01, 4.32876128e+01, 4.75081016e+01,
       5.21400829e+01, 5.72236766e+01, 6.28029144e+01, 6.89261210e+01,
       7.56463328e+01, 8.30217568e+01, 9.11162756e+01, 1.00000000e+02]))

In [232]:
# Calculate R2 scores
train_r2_ridge3 = ridge3.score(X_train3, y_train3)
test_r2_ridge3 = ridge3.score(X_test3, y_test3)

# Calculate RMSE
train_rmse_ridge3 = np.sqrt(metrics.mean_squared_error(y_train3, ridge3.predict(X_train3)))
test_rmse_ridge3 = np.sqrt(metrics.mean_squared_error(y_test3, ridge3.predict(X_test3)))

# Print the results
print("Ridge Model - Training R2 score:", train_r2_ridge3)
print("Ridge Model - Testing R2 score:", test_r2_ridge3)
print("Ridge Model - Training RMSE:", train_rmse_ridge3)
print("Ridge Model - Testing RMSE:", test_rmse_ridge3)

Ridge Model - Training R2 score: 0.9217340548543091
Ridge Model - Testing R2 score: 0.9207540375707537
Ridge Model - Training RMSE: 40079.50247499751
Ridge Model - Testing RMSE: 40256.40972282406


In [233]:
coefficients3 = ridge3.coef_

# Create a new DataFrame with predictors and coefficients
df_ridge3 = pd.DataFrame({"predictors": X3.columns, "coefficients": coefficients3})

# Sort the DataFrame ordered from largest to smallest coefficient by absolute values
df_ridge_sorted3 = df_ridge3.loc[df_ridge3["coefficients"].abs().sort_values(ascending=False).index]

# Reset the index
df_ridge_sorted3.reset_index(drop=True, inplace=True)

# Display all rows:
pd.set_option('display.max_rows', None)

#sorted dataframes
print("\nModel 3 Coefficients:")
print(df_ridge_sorted3)


Model 3 Coefficients:
                                            predictors   coefficients
0                                   flat_model_Terrace  221700.524458
1                                planning_area_Tanglin  220088.740682
2                            planning_area_Bukit Timah  180129.573907
3                          planning_area_Marine Parade  176823.487115
4                        full_flat_type_3 ROOM Terrace  147712.551261
5                              planning_area_Sembawang -117724.170014
6                              planning_area_Woodlands -108902.810163
7                                 planning_area_Outram  108545.775960
8                          planning_area_Bukit Panjang  -82529.390686
9                          planning_area_Choa Chu Kang  -79612.819159
10                                       market_hawker  -79594.976275
11                                planning_area_Bishan   77720.082598
12                            planning_area_Queenstown   75973.3113

## 03. Test Data Prediction

In [234]:
# Load data:
kaggle = pd.read_csv('./output/hdb_test_new.csv')

pd.set_option('display.max_columns', None)

# Check data:
print(kaggle.shape)
kaggle.head()

(16737, 48)


,Unnamed: 0,tranc_year,tranc_month,tranc_yearmonth,hdb_age,planning_area,commercial,market_hawker,multistorey_carpark,precinct_pavilion,max_floor_lvl,total_dwelling_units,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental,flat_type,flat_model,full_flat_type,mid_storey,floor_area_sqft,bus_interchange,bus_stop_nearest_distance,mrt_interchange,mrt_nearest_distance,mall_nearest_distance,mall_within_500m,mall_within_1km,mall_within_2km,hawker_food_stalls,hawker_market_stalls,hawker_nearest_distance,hawker_within_500m,hawker_within_2km,vacancy,pri_sch_affiliation,pri_sch_nearest_distance,cutoff_point,affiliation,sec_sch_nearest_dist
0,0,2012,11,2012-11,25,Yishun,1,0,0,0,12,132,0,0,0,92,40,0,0,0,0,0,0,0,4 ROOM,Simplified,4 ROOM Simplified,8,904.176,0,75.683952,0,686.660434,877.431572,0.0,2.0,4.0,56,123,687.576779,0.0,1.0,92,0,426.467910,218,0,156.322353
1,1,2019,8,2019-08,11,Jurong West,0,0,0,0,14,53,0,0,0,28,25,0,0,0,0,0,0,0,5 ROOM,Premium Apartment,5 ROOM Premium Apartment,5,1205.568,0,88.993058,0,169.478175,534.037705,0.0,1.0,3.0,72,94,2122.346226,0.0,0.0,45,0,439.756851,199,0,739.371688
2,2,2013,10,2013-10,33,Ang Mo Kio,0,0,0,0,12,218,0,0,191,22,1,1,0,0,0,0,3,0,3 ROOM,New Generation,3 ROOM New Generation,8,731.952,1,86.303575,0,694.220448,817.050453,0.0,2.0,3.0,50,100,152.287621,1.0,11.0,36,0,355.882207,245,0,305.071191
3,3,2017,10,2017-10,38,Woodlands,0,0,0,0,14,104,0,0,0,104,0,0,0,0,0,0,0,0,4 ROOM,New Generation,4 ROOM New Generation,2,1044.108,0,108.459039,0,1117.203587,1272.737194,0.0,0.0,3.0,52,112,501.892158,0.0,2.0,54,0,929.744711,188,0,433.454591
4,4,2016,8,2016-08,31,Bukit Batok,0,0,0,0,25,144,0,0,0,48,96,0,0,0,0,0,0,0,4 ROOM,Model A,4 ROOM Model A,17,1108.692,0,113.645431,0,987.976010,1070.963675,0.0,0.0,5.0,60,87,437.593564,1.0,2.0,40,0,309.926934,223,0,217.295361


### 06.01. Cleaning Test Data to match Model 3 Training Data

In [235]:
# Drop irrelevant columns
kaggle.drop(columns=['Unnamed: 0'], inplace=True)

In [236]:
# Create a copy of test dataframe
kaggle_ = kaggle.copy()

In [237]:
# Extracting specific columns for one-hot encoding, including 'full_flat_type', 'tranc_year', 'tranc_month', 'flat_model', 'flat_type', 'planning_area', 'tranc_yearmonth'
dummy = kaggle_[['full_flat_type', 'tranc_year', 'tranc_month', 'flat_model', 'flat_type', 'planning_area', 'tranc_yearmonth']]

In [238]:
# Convert 'tranc_month' to string type
dummy['tranc_month'] = dummy['tranc_month'].astype(str)


# Convert 'tranc_year' to string type
dummy['tranc_year'] = dummy['tranc_year'].astype(str)

/var/folders/fy/92rm3rl955997y4_8yf6zth00000gn/T/ipykernel_18879/3453044080.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dummy['tranc_month'] = dummy['tranc_month'].astype(str)
/var/folders/fy/92rm3rl955997y4_8yf6zth00000gn/T/ipykernel_18879/3453044080.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dummy['tranc_year'] = dummy['tranc_year'].astype(str)


In [239]:
# Convert categorical data to dummy variables using one-hot encoding
dummy = pd.get_dummies(dummy, drop_first=True)

# Check data after one-hot encoding
dummy.tail()

,full_flat_type_2 ROOM Improved,full_flat_type_2 ROOM Model A,full_flat_type_2 ROOM Premium Apartment,full_flat_type_2 ROOM Standard,full_flat_type_3 ROOM DBSS,full_flat_type_3 ROOM Improved,full_flat_type_3 ROOM Model A,full_flat_type_3 ROOM New Generation,full_flat_type_3 ROOM Premium Apartment,full_flat_type_3 ROOM Simplified,full_flat_type_3 ROOM Standard,full_flat_type_3 ROOM Terrace,full_flat_type_4 ROOM Adjoined flat,full_flat_type_4 ROOM DBSS,full_flat_type_4 ROOM Improved,full_flat_type_4 ROOM Model A,full_flat_type_4 ROOM Model A2,full_flat_type_4 ROOM New Generation,full_flat_type_4 ROOM Premium Apartment,full_flat_type_4 ROOM Premium Apartment Loft,full_flat_type_4 ROOM Simplified,full_flat_type_4 ROOM Standard,full_flat_type_4 ROOM Type S1,full_flat_type_5 ROOM Adjoined flat,full_flat_type_5 ROOM DBSS,full_flat_type_5 ROOM Improved,full_flat_type_5 ROOM Improved-Maisonette,full_flat_type_5 ROOM Model A,full_flat_type_5 ROOM Model A-Maisonette,full_flat_type_5 ROOM Premium Apartment,full_flat_type_5 ROOM Premium Apartment Loft,full_flat_type_5 ROOM Standard,full_flat_type_5 ROOM Type S2,full_flat_type_EXECUTIVE Adjoined flat,full_flat_type_EXECUTIVE Apartment,full_flat_type_EXECUTIVE Maisonette,full_flat_type_EXECUTIVE Premium Apartment,full_flat_type_EXECUTIVE Premium Maisonette,full_flat_type_MULTI-GENERATION Multi Generation,tranc_year_2013,tranc_year_2014,tranc_year_2015,tranc_year_2016,tranc_year_2017,tranc_year_2018,tranc_year_2019,tranc_year_2020,tranc_year_2021,tranc_month_10,tranc_month_11,tranc_month_12,tranc_month_2,tranc_month_3,tranc_month_4,tranc_month_5,tranc_month_6,tranc_month_7,tranc_month_8,tranc_month_9,flat_model_Apartment,flat_model_DBSS,flat_model_Improved,flat_model_Improved-Maisonette,flat_model_Maisonette,flat_model_Model A,flat_model_Model A-Maisonette,flat_model_Model A2,flat_model_Multi Generation,flat_model_New Generation,flat_model_Premium Apartment,flat_model_Premium Apartment Loft,flat_model_Premium Maisonette,flat_model_Simplified,flat_model_Standard,flat_model_Terrace,flat_model_Type S1,flat_model_Type S2,flat_type_2 ROOM,flat_type_3 ROOM,flat_type_4 ROOM,flat_type_5 ROOM,flat_type_EXECUTIVE,flat_type_MULTI-GENERATION,planning_area_Bedok,planning_area_Bishan,planning_area_Bukit Batok,planning_area_Bukit Merah,planning_area_Bukit Panjang,planning_area_Bukit Timah,planning_area_Changi,planning_area_Choa Chu Kang,planning_area_Clementi,planning_area_Downtown Core,planning_area_Geylang,planning_area_Hougang,planning_area_Jurong East,planning_area_Jurong West,planning_area_Kallang,planning_area_Marine Parade,planning_area_Novena,planning_area_Outram,planning_area_Pasir Ris,planning_area_Punggol,planning_area_Queenstown,planning_area_Rochor,planning_area_Sembawang,planning_area_Sengkang,planning_area_Serangoon,planning_area_Tampines,planning_area_Tanglin,planning_area_Toa Payoh,planning_area_Western Water Catchment,planning_area_Woodlands,planning_area_Yishun,tranc_yearmonth_2012-04,tranc_yearmonth_2012-05,tranc_yearmonth_2012-06,tranc_yearmonth_2012-07,tranc_yearmonth_2012-08,tranc_yearmonth_2012-09,tranc_yearmonth_2012-10,tranc_yearmonth_2012-11,tranc_yearmonth_2012-12,tranc_yearmonth_2013-01,tranc_yearmonth_2013-02,tranc_yearmonth_2013-03,tranc_yearmonth_2013-04,tranc_yearmonth_2013-05,tranc_yearmonth_2013-06,tranc_yearmonth_2013-07,tranc_yearmonth_2013-08,tranc_yearmonth_2013-09,tranc_yearmonth_2013-10,tranc_yearmonth_2013-11,tranc_yearmonth_2013-12,tranc_yearmonth_2014-01,tranc_yearmonth_2014-02,tranc_yearmonth_2014-03,tranc_yearmonth_2014-04,tranc_yearmonth_2014-05,tranc_yearmonth_2014-06,tranc_yearmonth_2014-07,tranc_yearmonth_2014-08,tranc_yearmonth_2014-09,tranc_yearmonth_2014-10,tranc_yearmonth_2014-11,tranc_yearmonth_2014-12,tranc_yearmonth_2015-01,tranc_yearmonth_2015-02,tranc_yearmonth_2015-03,tranc_yearmonth_2015-04,tranc_yearmonth_2015-05,tranc_yearmonth_2015-06,tranc_yearmonth_2015-07,tranc_yearmonth_2015-08,tranc_yearmonth_2015-09,tranc_yearmonth_2015-10,tranc_yearmonth

In [240]:
# Manually add the rest of the categorical features to dummy dataframe
dummy['affiliation'] = kaggle_['affiliation']
dummy['pri_sch_affiliation'] = kaggle_['pri_sch_affiliation']
dummy['mrt_interchange'] = kaggle_['mrt_interchange']
dummy['bus_interchange'] = kaggle_['bus_interchange']
dummy['precinct_pavilion'] = kaggle_['precinct_pavilion']
dummy['multistorey_carpark'] = kaggle_['multistorey_carpark']
dummy['market_hawker'] = kaggle_['market_hawker']
dummy['commercial'] = kaggle_['commercial']

In [241]:
# Check for null values
dummy.isna().sum()

full_flat_type_2 ROOM Improved                      0
full_flat_type_2 ROOM Model A                       0
full_flat_type_2 ROOM Premium Apartment             0
full_flat_type_2 ROOM Standard                      0
full_flat_type_3 ROOM DBSS                          0
full_flat_type_3 ROOM Improved                      0
full_flat_type_3 ROOM Model A                       0
full_flat_type_3 ROOM New Generation                0
full_flat_type_3 ROOM Premium Apartment             0
full_flat_type_3 ROOM Simplified                    0
full_flat_type_3 ROOM Standard                      0
full_flat_type_3 ROOM Terrace                       0
full_flat_type_4 ROOM Adjoined flat                 0
full_flat_type_4 ROOM DBSS                          0
full_flat_type_4 ROOM Improved                      0
full_flat_type_4 ROOM Model A                       0
full_flat_type_4 ROOM Model A2                      0
full_flat_type_4 ROOM New Generation                0
full_flat_type_4 ROOM Premiu

In [242]:
# Drop categorical features from original test dataframe
kaggle.drop(columns = ['full_flat_type', 'tranc_year', 'tranc_month', 'flat_model', 'flat_type', 'planning_area', 'tranc_yearmonth'], inplace=True)

In [243]:
# Drop categorical features from original test dataframe
kaggle.drop(columns = ['affiliation', 'pri_sch_affiliation', 'mrt_interchange', 'bus_interchange', 'precinct_pavilion', 'multistorey_carpark', 'market_hawker', 'commercial'], inplace = True)


In [244]:
kaggle.columns

Index(['hdb_age', 'max_floor_lvl', 'total_dwelling_units', '1room_sold',
       '2room_sold', '3room_sold', '4room_sold', '5room_sold', 'exec_sold',
       'multigen_sold', 'studio_apartment_sold', '1room_rental',
       '2room_rental', '3room_rental', 'other_room_rental', 'mid_storey',
       'floor_area_sqft', 'bus_stop_nearest_distance', 'mrt_nearest_distance',
       'mall_nearest_distance', 'mall_within_500m', 'mall_within_1km',
       'mall_within_2km', 'hawker_food_stalls', 'hawker_market_stalls',
       'hawker_nearest_distance', 'hawker_within_500m', 'hawker_within_2km',
       'vacancy', 'pri_sch_nearest_distance', 'cutoff_point',
       'sec_sch_nearest_dist'],
      dtype='object')

In [245]:
# Creating polynomial features up to the second degree with interactions for numerical data
poly_test = PolynomialFeatures(degree=2, include_bias=False)

# Transforming the numerical data with polynomial features
numerical_data = poly_test.fit_transform(kaggle)

In [246]:
## Creating a DataFrame for the polynomial features
# Get the feature names after polynomial transformation using the original DataFrame's column names
poly_feature_names = poly_test.get_feature_names_out(kaggle.columns)

# Create a DataFrame using numerical_data_poly_test and the obtained feature names
numerical_poly_test = pd.DataFrame(numerical_data, columns=poly_feature_names)

# Display the DataFrame
numerical_poly_test.head()

,hdb_age,max_floor_lvl,total_dwelling_units,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental,mid_storey,floor_area_sqft,bus_stop_nearest_distance,mrt_nearest_distance,mall_nearest_distance,mall_within_500m,mall_within_1km,mall_within_2km,hawker_food_stalls,hawker_market_stalls,hawker_nearest_distance,hawker_within_500m,hawker_within_2km,vacancy,pri_sch_nearest_distance,cutoff_point,sec_sch_nearest_dist,hdb_age^2,hdb_age max_floor_lvl,hdb_age total_dwelling_units,hdb_age 1room_sold,hdb_age 2room_sold,hdb_age 3room_sold,hdb_age 4room_sold,hdb_age 5room_sold,hdb_age exec_sold,hdb_age multigen_sold,hdb_age studio_apartment_sold,hdb_age 1room_rental,hdb_age 2room_rental,hdb_age 3room_rental,hdb_age other_room_rental,hdb_age mid_storey,hdb_age floor_area_sqft,hdb_age bus_stop_nearest_distance,hdb_age mrt_nearest_distance,hdb_age mall_nearest_distance,hdb_age mall_within_500m,hdb_age mall_within_1km,hdb_age mall_within_2km,hdb_age hawker_food_stalls,hdb_age hawker_market_stalls,hdb_age hawker_nearest_distance,hdb_age hawker_within_500m,hdb_age hawker_within_2km,hdb_age vacancy,hdb_age pri_sch_nearest_distance,hdb_age cutoff_point,hdb_age sec_sch_nearest_dist,max_floor_lvl^2,max_floor_lvl total_dwelling_units,max_floor_lvl 1room_sold,max_floor_lvl 2room_sold,max_floor_lvl 3room_sold,max_floor_lvl 4room_sold,max_floor_lvl 5room_sold,max_floor_lvl exec_sold,max_floor_lvl multigen_sold,max_floor_lvl studio_apartment_sold,max_floor_lvl 1room_rental,max_floor_lvl 2room_rental,max_floor_lvl 3room_rental,max_floor_lvl other_room_rental,max_floor_lvl mid_storey,max_floor_lvl floor_area_sqft,max_floor_lvl bus_stop_nearest_distance,max_floor_lvl mrt_nearest_distance,max_floor_lvl mall_nearest_distance,max_floor_lvl mall_within_500m,max_floor_lvl mall_within_1km,max_floor_lvl mall_within_2km,max_floor_lvl hawker_food_stalls,max_floor_lvl hawker_market_stalls,max_floor_lvl hawker_nearest_distance,max_floor_lvl hawker_within_500m,max_floor_lvl hawker_within_2km,max_floor_lvl vacancy,max_floor_lvl pri_sch_nearest_distance,max_floor_lvl cutoff_point,max_floor_lvl sec_sch_nearest_dist,total_dwelling_units^2,total_dwelling_units 1room_sold,total_dwelling_units 2room_sold,total_dwelling_units 3room_sold,total_dwelling_units 4room_sold,total_dwelling_units 5room_sold,total_dwelling_units exec_sold,total_dwelling_units multigen_sold,total_dwelling_units studio_apartment_sold,total_dwelling_units 1room_rental,total_dwelling_units 2room_rental,total_dwelling_units 3room_rental,total_dwelling_units other_room_rental,total_dwelling_units mid_storey,total_dwelling_units floor_area_sqft,total_dwelling_units bus_stop_nearest_distance,total_dwelling_units mrt_nearest_distance,total_dwelling_units mall_nearest_distance,total_dwelling_units mall_within_500m,total_dwelling_units mall_within_1km,total_dwelling_units mall_within_2km,total_dwelling_units hawker_food_stalls,total_dwelling_units hawker_market_stalls,total_dwelling_units hawker_nearest_distance,total_dwelling_units hawker_within_500m,total_dwelling_units hawker_within_2km,total_dwelling_units vacancy,total_dwelling_units pri_sch_nearest_distance,total_dwelling_units cutoff_point,total_dwelling_units sec_sch_nearest_dist,1room_sold^2,1room_sold 2room_sold,1room_sold 3room_sold,1room_sold 4room_sold,1room_sold 5room_sold,1room_sold exec_sold,1room_sold multigen_sold,1room_sold studio_apartment_sold,1room_sold 1room_rental,1room_sold 2room_rental,1room_sold 3room_rental,1room_sold other_room_rental,1room_sold mid_storey,1room_sold floor_area_sqft,1room_sold bus_stop_nearest_distance,1room_sold mrt_nearest_distance,1room_sold mall_nearest_distance,1room_sold mall_within_500m,1room_sold mall_within_1km,1room_sold mall_within_2km,1room_sold hawker_food_stalls,1room_sold hawker_market_stalls,1room_sold hawker_nearest_distance,1room_sold hawker_within_500m,1room_sold hawker_within_2km,1room_sold vacancy,1

In [247]:
# Check number of categorical columns in training data for optimal model 3
len(model_3_binary_columns)

235

In [248]:
# Check shape of categorical columns in test data
dummy.shape

(16737, 231)

In [249]:
# Get column names of both dataframes
dummy_columns = set(dummy.columns)
model_3_binary_columns_1 = set(model_3_binary_columns)

# Find column names that are in dummy but not in model_3_binary
in_dummy_not_in_model_3_binary = dummy_columns - model_3_binary_columns_1

# Find column names that are in model_3_binary but not in dummy
in_model_3_binary_not_in_dummy = model_3_binary_columns_1 - dummy_columns

print("Columns in dummy but not in model_3_binary: ", in_dummy_not_in_model_3_binary)
print("Columns in model_3_binary but not in dummy: ", in_model_3_binary_not_in_dummy)

Columns in dummy but not in model_3_binary:  set()
Columns in model_3_binary but not in dummy:  {'full_flat_type_4 ROOM Terrace', 'full_flat_type_2 ROOM DBSS', 'flat_model_Adjoined flat', 'full_flat_type_2 ROOM 2-room'}


In [250]:
# Add missing columns to 'dummy' with all values set to False
dummy = dummy.reindex(columns = model_3_binary_columns, fill_value=False)

In [251]:
# Check number categorical columns in test data
len(dummy.columns)

235

In [252]:
# Check number of categorical columns in training data for optimal model 3
len(model_3_binary_columns)

235

In [253]:
# Scale the test data using the same mean and standard deviation obtained from training model 3
numerical_poly_test_scaled = ss3.fit_transform(numerical_poly_test)

In [254]:
# Converting scaled data from array to dataframe
numerical = pd.DataFrame(numerical_poly_test_scaled, columns=numerical_poly_test.columns)

In [255]:
for item in model_3_numerical_columns:
    if item not in numerical:
        print(item)

2room_sold 1room_sold
4room_sold 1room_sold
5room_sold 1room_sold
exec_sold 1room_sold
multigen_sold 1room_sold
studio_apartment_sold 1room_sold
2room_rental 1room_sold
3room_rental 1room_sold
other_room_rental 1room_sold
mall_within_500m 1room_sold
mall_within_1km 1room_sold


In [256]:
numerical.rename(columns = {"1room_sold 2room_sold": "2room_sold 1room_sold",
"1room_sold 4room_sold": "4room_sold 1room_sold",
"1room_sold 5room_sold": "5room_sold 1room_sold",
"1room_sold exec_sold": "exec_sold 1room_sold",
"1room_sold multigen_sold": "multigen_sold 1room_sold",
"1room_sold studio_apartment_sold": "studio_apartment_sold 1room_sold",
"1room_sold 2room_rental": "2room_rental 1room_sold",
"1room_sold 3room_rental": "3room_rental 1room_sold",
"1room_sold other_room_rental": "other_room_rental 1room_sold",
"1room_sold mall_within_500m": "mall_within_500m 1room_sold",
"1room_sold mall_within_1km": "mall_within_1km 1room_sold"}, inplace=True)

In [257]:
for item in model_3_numerical_columns:
    if item not in numerical:
        print(item)

In [258]:
# Match numerical test features to numerical training features
numerical = numerical[model_3_numerical_columns]

In [259]:
# Check number of numerical features columns in test data 
len(numerical.columns)

267

In [260]:
# Check number of numerical features columns in training data for optimal model 3
len(model_3_numerical_columns)

267

In [261]:
# Concatenate cleaned test data
preprocessed_test_df = pd.concat([numerical, 
                           dummy], axis=1)

# Display the combined DataFrame
preprocessed_test_df.head()

,2room_sold,3room_sold,floor_area_sqft,hawker_food_stalls,vacancy,hdb_age^2,hdb_age max_floor_lvl,hdb_age 2room_sold,hdb_age 4room_sold,hdb_age 1room_rental,hdb_age mid_storey,hdb_age floor_area_sqft,hdb_age bus_stop_nearest_distance,hdb_age mrt_nearest_distance,hdb_age mall_nearest_distance,hdb_age mall_within_1km,hdb_age mall_within_2km,hdb_age hawker_food_stalls,hdb_age hawker_market_stalls,hdb_age hawker_nearest_distance,hdb_age hawker_within_500m,hdb_age vacancy,hdb_age pri_sch_nearest_distance,hdb_age sec_sch_nearest_dist,hdb_age 1room_sold,max_floor_lvl 4room_sold,max_floor_lvl 5room_sold,max_floor_lvl floor_area_sqft,max_floor_lvl bus_stop_nearest_distance,max_floor_lvl mall_nearest_distance,max_floor_lvl mall_within_500m,max_floor_lvl mall_within_1km,max_floor_lvl mall_within_2km,max_floor_lvl hawker_food_stalls,max_floor_lvl hawker_market_stalls,max_floor_lvl hawker_within_500m,max_floor_lvl hawker_within_2km,max_floor_lvl vacancy,max_floor_lvl pri_sch_nearest_distance,max_floor_lvl sec_sch_nearest_dist,total_dwelling_units floor_area_sqft,total_dwelling_units bus_stop_nearest_distance,total_dwelling_units mrt_nearest_distance,total_dwelling_units mall_nearest_distance,total_dwelling_units mall_within_2km,total_dwelling_units hawker_food_stalls,total_dwelling_units hawker_nearest_distance,total_dwelling_units pri_sch_nearest_distance,total_dwelling_units cutoff_point,total_dwelling_units sec_sch_nearest_dist,2room_sold 3room_sold,2room_sold 4room_sold,2room_sold 5room_sold,2room_sold exec_sold,2room_sold multigen_sold,2room_sold 2room_rental,2room_sold 3room_rental,2room_sold other_room_rental,2room_sold mall_within_500m,2room_sold mall_within_2km,2room_sold hawker_market_stalls,2room_sold hawker_nearest_distance,2room_sold 1room_sold,3room_sold 4room_sold,3room_sold 5room_sold,3room_sold exec_sold,3room_sold multigen_sold,3room_sold studio_apartment_sold,3room_sold 1room_rental,3room_sold other_room_rental,3room_sold mall_within_500m,3room_sold mall_within_1km,3room_sold mall_within_2km,3room_sold hawker_nearest_distance,3room_sold hawker_within_500m,4room_sold 5room_sold,4room_sold exec_sold,4room_sold studio_apartment_sold,4room_sold 2room_rental,4room_sold 3room_rental,4room_sold bus_stop_nearest_distance,4room_sold mrt_nearest_distance,4room_sold mall_nearest_distance,4room_sold mall_within_500m,4room_sold mall_within_1km,4room_sold mall_within_2km,4room_sold hawker_food_stalls,4room_sold hawker_market_stalls,4room_sold hawker_nearest_distance,4room_sold hawker_within_500m,4room_sold hawker_within_2km,4room_sold vacancy,4room_sold pri_sch_nearest_distance,4room_sold sec_sch_nearest_dist,4room_sold 1room_sold,5room_sold exec_sold,5room_sold multigen_sold,5room_sold studio_apartment_sold,5room_sold 1room_rental,5room_sold 2room_rental,5room_sold 3room_rental,5room_sold other_room_rental,5room_sold bus_stop_nearest_distance,5room_sold mrt_nearest_distance,5room_sold mall_nearest_distance,5room_sold mall_within_500m,5room_sold mall_within_1km,5room_sold hawker_food_stalls,5room_sold hawker_market_stalls,5room_sold hawker_nearest_distance,5room_sold hawker_within_500m,5room_sold hawker_within_2km,5room_sold pri_sch_nearest_distance,5room_sold sec_sch_nearest_dist,5room_sold 1room_sold,exec_sold multigen_sold,exec_sold studio_apartment_sold,exec_sold 1room_rental,exec_sold 2room_rental,exec_sold 3room_rental,exec_sold other_room_rental,exec_sold floor_area_sqft,exec_sold mall_within_500m,exec_sold mall_within_1km,exec_sold hawker_market_stalls,exec_sold hawker_nearest_distance,exec_sold hawker_within_500m,exec_sold hawker_within_2km,exec_sold 1room_sold,multigen_sold studio_apartment_sold,multigen_sold 1room_rental,multigen_sold 2room_rental,multigen_sold 3room_rental,multigen_sold other_room_rental,multigen_sold floor_area_sqft,multigen_sold mall_nearest_distance,multigen_sold hawker_market_stalls,multigen_sold 1room_sold,studio_apartment_sold 1room_rental,studio_apartment_sold 2room_rental,studio_apartment_sold 3roo

### 06.02. Run predictions for test data

In [262]:
# Get predictions on the test data
predictions_test = ridge3.predict(preprocessed_test_df)

In [263]:
# Load origina; data to extract id columns
id = pd.read_csv('./data/test.csv')

/var/folders/fy/92rm3rl955997y4_8yf6zth00000gn/T/ipykernel_18879/1129173912.py:2: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  id = pd.read_csv('./data/test.csv')


In [264]:
# extract id columns
Id = id['id']
Id.tail()

16732     23347
16733     54003
16734    128921
16735     69352
16736    146210
Name: id, dtype: int64

In [265]:
# check number of rows in predicted data
len(predictions_test)

16737

In [266]:
# Stick'id' to predictions_test side by side
result_df = pd.concat([Id, pd.Series(predictions_test, name='Predicted')], axis=1)

In [267]:
result_df.head()

,id,Predicted
0,114982,372964.369168
1,95653,488818.228849
2,40303,339135.367996
3,109506,261729.556577
4,100149,442459.809091


In [268]:
# Rename the 'id' column to 'Id'
result_df = result_df.rename(columns={'id': 'Id'})

# Display the updated DataFrame
print(result_df.head())

       Id      Predicted
0  114982  372964.369168
1   95653  488818.228849
2   40303  339135.367996
3  109506  261729.556577
4  100149  442459.809091


### 06.03. Export predicted test results for submission

In [269]:
# Export predicted results
import os  # to work with files/directories

# Define the output folder path
output_folder_path = '../project_2/output'

# Check if the output folder exists, and create it if not
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# Save Model_3 CSV file within the 'output' folder
result_df.to_csv(os.path.join(output_folder_path, 'Kaggle_submission.csv'), index=False)